<a href="https://colab.research.google.com/github/martadiridolfi/TESI/blob/main/LDNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import modules**

In [4]:
#!/usr/bin/env python3
#%% Import modules
import sys
sys.path.insert(0, '/u/sw/toolchains/gcc-glibc/11.2.0/pkgs/scipy/1.7.0/lib/python3.9/site-packages')

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
# We configure TensorFlow to work in double precision
tf.keras.backend.set_floatx('float64')

from google.colab import files
uploaded = files.upload()

import utils
import optimization
import os


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Set some hyperparameters**

In [4]:

#%% Set some hyperparameters
dt = 2.27e-03
dt_base = 1.04e-1
num_latent_states = 9


# **Define problem**

In [5]:
#%% Define problem
problem = {
    "space": {
        "dimension" : 3
    },
    #"input_parameters": [
    #    { "name": "diameter" }
    #],
    #"input_signals": [
    #    { "name": "impulse" }
    #],
    "output_fields": [
        { "name": "u" }
    ]
}

normalization = {
    'space': { 'min' : [0], 'max' : [+100.0]},
    'time': { 'time_constant' : dt_base },
    #'input_parameters': {
    #    'diameter': { 'min': 1.0 , 'max': 10.0 },
    #},
    #'input_signals': {
    #    'impulse': { 'min':   0.0 , 'max': 1.0},
    #},
    'output_fields': {
        'u': { 'min': 0.0, "max": 1.0 }
    }
}




#**Dataset**

In [13]:
#%% Dataset
data_set_path = '/content/data/LA_healthy/'

import scipy.io
mat_1 = scipy.io.loadmat('/content/data/LA_healthy/sample_1.mat')
#print(mat_1.keys())

dataset_train = utils.LAhealthy_create_dataset(data_set_path, 0, 1)
dataset_valid = utils.LAhealthy_create_dataset(data_set_path, 0, 1)
dataset_tests = utils.LAhealthy_create_dataset(data_set_path, 0, 1)

# For reproducibility (delete if you want to test other random initializations)
np.random.seed(0)
tf.random.set_seed(0)

n_subsample = 2000

# We re-sample the time transients with timestep dt and we rescale each variable between -1 and 1.
utils.process_dataset(dataset_train, problem, normalization, dt = dt, num_points_subsample = n_subsample)
utils.process_dataset(dataset_valid, problem, normalization, dt = dt, num_points_subsample = n_subsample)
utils.process_dataset(dataset_tests, problem, normalization, dt = dt)


loading dataset 1
loading dataset 1
loading dataset 1


#**Model construction**

In [1]:
#%% Model construction
'''
# dynamics network
input_shape = (num_latent_states,)# + len(problem['input_signals']) + len(problem['input_parameters']),)
NNdyn = tf.keras.Sequential([
            tf.keras.layers.Dense(28, activation = tf.nn.tanh, input_shape = input_shape),
            tf.keras.layers.Dense(28, activation = tf.nn.tanh),
            tf.keras.layers.Dense(28, activation = tf.nn.tanh),
            tf.keras.layers.Dense(28, activation = tf.nn.tanh),
            tf.keras.layers.Dense(28, activation = tf.nn.tanh),
            tf.keras.layers.Dense(28, activation = tf.nn.tanh),
            tf.keras.layers.Dense(28, activation = tf.nn.tanh),
            tf.keras.layers.Dense(28, activation = tf.nn.tanh),
            tf.keras.layers.Dense(28, activation = tf.nn.tanh),
            tf.keras.layers.Dense(28, activation = tf.nn.tanh),
            tf.keras.layers.Dense(num_latent_states)
        ])
NNdyn.summary()

# reconstruction network
input_shape = (None, None, num_latent_states + problem['space']['dimension'])
NNrec = tf.keras.Sequential([
            tf.keras.layers.Dense(22, activation = tf.nn.tanh, input_shape = input_shape),
            tf.keras.layers.Dense(22, activation = tf.nn.tanh),
            tf.keras.layers.Dense(22, activation = tf.nn.tanh),
            tf.keras.layers.Dense(22, activation = tf.nn.tanh),
            tf.keras.layers.Dense(22, activation = tf.nn.tanh),
            tf.keras.layers.Dense(len(problem['output_fields']))
        ])
NNrec.summary()
'''

# Carica i modelli allenati
NNdyn = tf.keras.models.load_model(f'/content/LDNets-main/src/Models/NNdyn_model_{n_subsample}.h5')
NNrec = tf.keras.models.load_model(f'/content/LDNets-main/src/Models/NNrec_model_{n_subsample}.h5')

NameError: name 'tf' is not defined

In [ ]:
def evolve_dynamics(dataset):
    # intial condition
    state = tf.zeros((dataset['num_samples'], num_latent_states), dtype=tf.float64) #Inizializzazione dello stato latente con un tensore di zeri
    state_history = tf.TensorArray(tf.float64, size = dataset['num_times']) #TensorArray che tiene traccia della storia degli stati latenti nel tempo. dim = n. passi temporali
    state_history = state_history.write(0, state) #Registrazione stato iniziale (t=0) nel TensorArray
    dt_ref = normalization['time']['time_constant'] #Riferimento temporale per la normalizzazione di dt

    # time integration
    for i in tf.range(dataset['num_times'] - 1):
        state = state + dt/dt_ref * NNdyn(tf.concat([state], axis = -1)) # dataset['inp_parameters'], dataset['inp_signals'][:,i,:]], axis = -1)) #Aggiornamento dello stato corrente mediante l'aggiunta della variazione
        state_history = state_history.write(i + 1, state) #Memorizzazione del nuovo stato nel TensorArray

    return tf.transpose(state_history.stack(), perm=(1,0,2)) #Restituzione dell'intera storia degli stati attraverso il tempo per tutti i campioni simulati.

def reconstruct_output(dataset, states):
    states_expanded = tf.broadcast_to(tf.expand_dims(states, axis = 2), #expand_dims aggiunge una nuova dimensione all'indice 2 del tensore states
        [dataset['num_samples'], dataset['num_times'], dataset['num_points'], num_latent_states]) #Il broadcasting è un'operazione che espande un tensore lungo una dimensione senza dover replicare fisicamente i dati, consentendo di operare su tensori con forme compatibili.
    return NNrec(tf.concat([states_expanded, dataset['points_full']], axis = 3)) #Rete neurale applicata alla concatenazione di states_expanded con dataset[points_full] restituendo il campo fisico

def LDNet(dataset):
    states = evolve_dynamics(dataset) #evoluzione temporale degli stati latenti
    return reconstruct_output(dataset, states) #ricostruzione dell'output

# Loss function
def MSE(dataset):
    out_fields = LDNet(dataset)
    error = out_fields - dataset['out_fields']
    return tf.reduce_mean(tf.square(error))

def weights_reg(NN):
    return sum([tf.reduce_mean(tf.square(lay.kernel)) for lay in NN.layers])/len(NN.layers)

alpha_reg = 4.81e-4
def loss(): return MSE(dataset_train) + alpha_reg * (weights_reg(NNdyn) + weights_reg(NNrec))
def MSE_valid(): return MSE(dataset_valid)

#**Training**

In [ ]:
#%% Training
trainable_variables = NNdyn.variables + NNrec.variables
opt = optimization.OptimizationProblem(trainable_variables, loss, MSE_valid)

#num_epochs_Adam = 100
num_epochs_BFGS = 1000

#print('training (Adam)...')
#opt.optimize_keras(num_epochs_Adam, tf.keras.optimizers.Adam(learning_rate=1e-2))
print('training (BFGS)...')
opt.optimize_BFGS(num_epochs_BFGS)


fig, axs = plt.subplots(1, 1)
axs.loglog(opt.iterations_history, opt.loss_train_history, 'o-', label = 'training loss')
axs.loglog(opt.iterations_history, opt.loss_valid_history, 'o-', label = 'validation loss')
#axs.axvline(num_epochs_Adam)
axs.set_xlabel('epochs'), plt.ylabel('loss')
axs.legend()
fig.savefig('training_loss.png')

# Salva il modello allenato
NNdyn.save(os.path.join('/content/LDNets-main/src/Models/', f'NNdyn_model_{n_subsample}.h5'))
NNrec.save(os.path.join('/content/LDNets-main/src/Models/', f'NNrec_model_{n_subsample}.h5'))

'''
# Carica i modelli allenati
NNdyn = tf.keras.models.load_model(f'/home/martadr/Scrivania/TESI/LDNets/LDNets-main/src/Models/NNdyn_model_{n_subsample}.h5')
NNrec = tf.keras.models.load_model(f'/home/martadr/Scrivania/TESI/LDNets/LDNets-main/src/Models/NNrec_model_{n_subsample}.h5')
'''

#**Testing**

In [ ]:
#%% Testing
# Compute predictions.
out_fields = LDNet(dataset_tests)
# Since the models work with normalized data, we map back the outputs into the original ranges.
out_fields_FOM = utils.denormalize_output(dataset_tests['out_fields'], problem, normalization).numpy()
out_fields_ROM = utils.denormalize_output(out_fields                 , problem, normalization).numpy()

NRMSE = np.sqrt(np.mean(np.square(out_fields_ROM - out_fields_FOM))) / (np.max(out_fields_FOM) - np.min(out_fields_FOM))

import scipy.stats
R_coeff = scipy.stats.pearsonr(np.reshape(out_fields_ROM, (-1,)), np.reshape(out_fields_FOM, (-1,)))

print('Normalized RMSE:       %1.3e' % NRMSE)
print('Pearson dissimilarity: %1.3e' % (1 - R_coeff[0]))

#**Postprocessing**

In [ ]:
#%% Postprocessing

import pyvista as pv

#print(type(dataset_tests['points']))

coordinates = dataset_tests['points']
#print(out_fields_ROM.shape[1])


for t in range(out_fields_ROM.shape[1]):

    # Estrai i valori del potenziale per il tempo t
    potenziali_ROM = out_fields_ROM[0, t, :, 0]

    #potenziali_ROM_max_t = np.max(potenziali_ROM)
    #potenziali_ROM_max.append(potenziali_ROM_max_t)

    # Crea una griglia PyVista con le coordinate
    grid_ROM = pv.PolyData(coordinates)
    grid_ROM['Potential'] = potenziali_ROM  # Aggiungi i valori del potenziale come dati scalari

    # Specifica la cartella di destinazione (puoi usare un percorso assoluto o relativo)
    output_folder_ROM = f'/home/martadr/Scrivania/TESI/LDNets/results/LA_healthy_sample_1_ROM_{n_subsample}/'

    # Crea la cartella se non esiste
    if not os.path.exists(output_folder_ROM):
        os.makedirs(output_folder_ROM)


    grid_ROM.save(os.path.join(output_folder_ROM, f'data_time_{t}.vtk'))

    '''
    # Estrai i valori del potenziale per il tempo t
    potenziali_FOM = out_fields_FOM[0, t, :, 0]

    #print(potenziali.shape)

    #potenziali_FOM_max_t = np.max(potenziali_FOM)
    #potenziali_FOM_max.append(potenziali_FOM_max_t)

    # Crea una griglia PyVista con le coordinate
    grid_FOM = pv.PolyData(coordinates)
    grid_FOM['Potential'] = potenziali_FOM  # Aggiungi i valori del potenziale come dati scalari

    # Specifica la cartella di destinazione (puoi usare un percorso assoluto o relativo)
    output_folder_FOM = f'/home/martadr/Scrivania/TESI/LDNets/results/LA_healthy_sample_1_FOM_{n_subsample}/'

    # Crea la cartella se non esiste
    if not os.path.exists(output_folder_FOM):
        os.makedirs(output_folder_FOM)

    # Costruisci il percorso completo per il file .vtk
    filename_vtk_FOM = os.path.join(output_folder_FOM, f'data_time_{t}.vtk')

    grid_FOM.save(filename_vtk_FOM)
    '''


#potenziali_ROM_max = np.array(potenziali_ROM_max)

#print(np.max(potenziali_ROM_max))

#potenziali_FOM_max = np.array(potenziali_FOM_max)

#print(np.max(potenziali_FOM_max))